In [2]:
import pandas as pd
import numpy as np
import requests
import re
import urllib.parse
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from pathlib import Path
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

In [31]:
#Mcdonalds Big Mac Index
Mcdf = pd.read_json("big-mac-index-by-state-2024.json")  # Read JSON file directly
Mcdf= Mcdf.rename(columns={"BigMacIndexByState2023": "McdonaldsBigMac2023"})

In [39]:
#Minimum Wage data
#excel to dataframe
minwagedf = pd.read_csv('minimum-wage-2024-by-state2.csv')
minwagedf = minwagedf.sort_values(by='state').reset_index(drop=True)


In [28]:
#median/ average income
states=[]
averages= []
medians= []

url = "https://www.sofi.com/learn/content/average-salary-in-us/"
r = requests.get(url)
text = r.text
soup= BeautifulSoup(r.text)
trs = soup.find_all('tr')

for row in trs:  
    cols = row.find_all("td")  # Get all columns in the row
    if len(cols) >= 3:  # Ensure it has enough columns
        states.append(cols[0].text.strip())
        averages.append(int(cols[1].text.replace("$", "").replace(",", "")))
        medians.append(int(cols[2].text.replace("$", "").replace(",", "")))
medavgdf = pd.DataFrame({'State':states, 'Average':averages, 'Median':medians})
medavgdf= medavgdf.rename(columns={"State": "state"})


In [29]:
#cost of living
livingcostdf = pd.read_json("cost-of-living-index-by-state-2025.json")  # Read JSON file directly
livingcostdf = livingcostdf.drop(columns=['stateFlagCode'])
livingcostdf = livingcostdf.rename(columns={
    "CostOfLivingIndex_CostOfLivingIndex_num_2024": "CostOfLivingIndex",
    "CostOfLivingIndex_GroceriesCostsIndex_num_2024": "GroceriesCostsIndex",
    "CostOfLivingIndex_HousingCostsIndex_num_2024": "HousingCostsIndex",
    "CostOfLivingIndex_TransportationCostsIndex_num_2024": "TransportationCostsIndex",
    "CostOfLivingIndex_UtilitiesCostsIndex_num_2024": "UtilitiesCostsIndex",
    "CostOfLivingIndex_HealthCostsIndex_num_2024": "HealthCareCostsIndex",
    "CostOfLivingIndex_MiscCostsIndex_num_2024": "MiscellaneousGoodsServicesCostsIndex"
})
livingcostdf= livingcostdf.sort_values(by='state').reset_index(drop=True)


In [30]:
#fast food prices
fastfooddf = pd.read_json("fast-food-prices-by-state-2025.json")
fastfooddf = fastfooddf.drop(columns=['stateFlagCode'])
fastfooddf = fastfooddf.rename(columns={
    "FastFoodPricesDominosMedCheese": "DominosMedCheese",
    "FastFoodPricesMcDonaldsHappyMeal": "McDonaldsHappyMeal",
    "FastFoodPricesMcDonaldsBigMac": "McDonaldsBigMac2025",
    "FastFoodPricesChickfilAChickenSandwich": "ChickfilAChickenSandwich",
    "FastFoodPricesTacoBellComboMeal": "TacoBellComboMeal"
})
fastfooddf= fastfooddf.sort_values(by='state').reset_index(drop=True)

In [41]:
#combine all df into one on state
combined_df = medavgdf.merge(livingcostdf, on='state', how='outer') \
                      .merge(minwagedf, on='state', how='outer') \
                      .merge(Mcdf, on='state', how='outer') \
                      .merge(fastfooddf, on='state', how='outer')

combined_df
combined_df.to_csv('fast_food_analysis.csv', index=False)